In [ ]:
import datetime
import os,sys
import matplotlib.pyplot as plt
import h5py as h5
import numpy as np
import attr
import re
import shutil
import io
import matplotlib
#sys.exit()
from timeit import default_timer as timer

'''This python script does the following:
    1. initializes path to the trial that we are doing, within the general 
        aspects of the input deck we are changing
    2. gets the slice data file paths and, thus, the file names
    3. creates graphs
    4. creates txt files with the data that creates the graphs
'''

run_dir = 'diagGaussian_yoffum_00.0_tiltdeg_0000_20210702_101231'

# write path to where the python files are
input_path = '/bdata/cphillips/jupyternotebooks/recreateCcode/'

#path = os.path.join(input_path, 'graphs') 
#os.mkdir(path)

hfiles = os.listdir(input_path+run_dir+'/hdf5/')

def getHpaths(file_name):
    dir_path = input_path+run_dir+'/hdf5/'+file_name
    return dir_path

# make list of paths to the hdf5 files
file_names = [getHpaths(a) for a in hfiles]
#print(file_names)


# Let's loop over all the .h5 files 

#from the testingawacode, what is w, Nx, Ny, and 2a?
w = 12.7e-3
Nx = 128
a_2 = 2.5e-3
Ny = 64
num_ticks = 5

x_dim = Nx+1
y_dim = Ny+1
x_nums = np.arange(0,x_dim,x_dim/num_ticks)
y_nums = np.arange(0,y_dim,y_dim/num_ticks)

# Units are mm
xticks = [x_nums[i]*w*1e3/(Nx+1) for i in np.arange(0,num_ticks)] 
yticks = [y_nums[i]*a_2*1e3/(Ny+1) for i in np.arange(0,num_ticks)]

for i in range(0,len(file_names)):
    # first, read the .h5 file
    f_tstep = h5.File(file_names[i],"r")
    # make list of attributes inside file (first: name, second: data name)
#f_datanames = list(f_tstep.keys())
    # get to the E_z data
    dset_data = f_tstep['data']
    dset_timename = list(dset_data.keys())
    dset_time = dset_data[dset_timename[0]]
    dset_field = dset_time['fields']
    dset_efld = dset_field['E']
    dset_ez = dset_efld['z']
#    print(np.shape(dset_ez))
    fig, ax = plt.subplots()
    im = ax.imshow(dset_ez[2])
    ax.set_xticks(np.arange(0,x_dim,x_dim/num_ticks))
    ax.set_xticklabels(xticks)
    ax.set_yticks(np.arange(0,y_dim,y_dim/num_ticks))
    ax.set_yticklabels(yticks)
    plt.xlabel(r'x (mm)')
    plt.ylabel(r'y (mm)')
    ax.set_title(r"Plot for E_z ts="+str((i+1)*20))
    fig.tight_layout()
    plt.savefig(run_dir+'/graphs/'+hfiles[i]+'.png')
    plt.close()
    
    
    c = 3.e8

tstepnum = 160 #must be part of the timesteps that the original code output

f_tstep = h5.File(file_names[int(tstepnum/20)],"r")
# make list of attributes inside file (first: name, second: data name)
#f_datanames = list(f_tstep.keys())
# get to the E_z data
dset_data = f_tstep['data']
dset_timename = list(dset_data.keys())
dset_time = dset_data[dset_timename[0]]
dset_field = dset_time['fields']
dset_bfld = dset_field['B']
dset_bx = np.array(dset_bfld['x'])
dset_by = np.array(dset_bfld['y'])
dset_efld = dset_field['E']
dset_ex = np.array(dset_efld['x'])
dset_ey = np.array(dset_efld['y'])
calc_fx = dset_ex - c*dset_by
calc_fy = dset_ey + c*dset_bx
print(np.shape(calc_fx))
fig, ax = plt.subplots()
im = ax.imshow(calc_fx[2])
ax.set_xticks(np.arange(0,x_dim,x_dim/num_ticks))
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,y_dim,y_dim/num_ticks))
ax.set_yticklabels(yticks)
plt.xlabel(r'x (mm)')
plt.ylabel(r'y (mm)')
ax.set_title(r"Plot for Fx")
fig.tight_layout()
plt.savefig(run_dir+'/graphs/'+str(int(tstepnum))+'Fx3D.png')
#plt.savefig(run_dir+'/graphs/'+hfiles[i]+'.png')


fig, ax = plt.subplots()
im = ax.imshow(calc_fy[2])
x_dim = dset_ez.shape[0]
y_dim = dset_ez.shape[1]
ax.set_xticks(np.arange(0,x_dim,x_dim/num_ticks))
ax.set_xticklabels(xticks)
ax.set_yticks(np.arange(0,y_dim,y_dim/num_ticks))
ax.set_yticklabels(yticks)
plt.xlabel(r'x (mm)')
plt.ylabel(r'y (mm)')
ax.set_title(r"Plot for Fy")
fig.tight_layout()
plt.savefig(run_dir+'/graphs/'+str(int(tstepnum))+'Fy3D.png')